# **Clasificación de Texto con Transformers (BERT y DistilBERT) en PyTorch**
## **Dataset: IMDB Movie Reviews**

**Objetivo del Script:**
Este script presenta un flujo de trabajo completo para construir, entrenar, evaluar y utilizar modelos de Transformers (BERT y DistilBERT) para la clasificación de texto utilizando **PyTorch**. Utilizaremos el dataset "IMDB" para la clasificación de opiniones de películas (positivo/negativo).

**Estructura del Script:**
1.  Carga de las librerías necesarias.
2.  Definición de constantes y configuración.
3.  Carga y preprocesamiento del dataset IMDB.
4.  Definición de los modelos (BERT y DistilBERT).
5.  Definición de las funciones de entrenamiento y evaluación.
6.  Entrenamiento y evaluación de los modelos.
7.  Inferencia con el mejor modelo.
8.  Guardado del mejor modelo.

In [1]:
!pip install torchtext==0.16.0 torch==2.1.0 transformers==4.31.0
!pip install numpy==1.25.0 portalocker>=2.0.0

In [2]:
# 1. Carga de las Librerías
# ------------------------------------------------------------------------------

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

# Hugging Face Transformers para modelos y tokenizers
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

# TorchText para cargar el dataset IMDB
import torchtext
# Descomentar la siguiente línea si es la primera vez que se usa torchtext o da error
# torchtext.disable_torchtext_deprecation_warning()
from torchtext.datasets import IMDB

# Otras utilidades
import numpy as np
import os
import datetime
import random
from tqdm.auto import tqdm # Para barras de progreso

# Comprobación de versiones
print(f"PyTorch Version: {torch.__version__}")
print(f"TorchText Version: {torchtext.__version__}")

# Determinar el dispositivo (GPU si está disponible, sino CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsando dispositivo: {device}")

PyTorch Version: 2.1.0+cu121
TorchText Version: 0.16.0+cpu

Usando dispositivo: cuda


In [3]:
# 2. Definición de Constantes y Configuración
# ------------------------------------------------------------------------------

# Constantes del Modelo
MAX_LEN = 256  # Longitud máxima de las secuencias para el tokenizer
BATCH_SIZE = 16 # Tamaño del lote (batch size). Reducir si hay errores de memoria (OOM).
EPOCHS = 3      # Número de épocas de entrenamiento
LEARNING_RATE = 2e-5 # Tasa de aprendizaje para el optimizador AdamW

# Para reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device.type == 'cuda':
    torch.cuda.manual_seed_all(SEED)

In [4]:
# 3. Carga y Preprocesamiento del Dataset IMDB
# ------------------------------------------------------------------------------
print("\n--- Cargando y Preprocesando el Dataset IMDB ---")

# Nota: torchtext.datasets.IMDB devuelve un iterador. Lo convertiremos a una lista.
# El dataset ya viene pre-dividido en 'train' y 'test'.
train_iter, test_iter = IMDB(split=('train', 'test'))

# Convertir los iteradores a listas para facilitar el manejo
# Formato: [(label, text), ...] donde label es 1 para 'neg' y 2 para 'pos'.
train_data = list(train_iter)
test_data = list(test_iter)

# Mapeamos las etiquetas a 0 (negativo) y 1 (positivo)
def preprocess_data(data):
    processed = []
    for label, text in data:
        # '1' es neg, '2' es pos. Lo mapeamos a 0 y 1.
        processed.append((text, 0 if label == 1 else 1))
    return processed

train_texts, train_labels = zip(*preprocess_data(train_data))
test_texts, test_labels = zip(*preprocess_data(test_data))

print(f"Dataset IMDB cargado.")
print(f"Ejemplos de entrenamiento: {len(train_texts)}")
print(f"Ejemplos de prueba: {len(test_texts)}")
print(f"Ejemplo de texto: '{train_texts[0][:100]}...'")
print(f"Etiqueta correspondiente: {'Positivo' if train_labels[0] == 1 else 'Negativo'}")


def create_dataloader(texts, labels, tokenizer, max_len, batch_size):
    """Tokeniza los textos y crea un DataLoader de PyTorch."""
    print(f"\nTokenizando datos con {tokenizer.__class__.__name__}...")

    # `encode_plus` se encarga de tokenizar, añadir tokens especiales ([CLS], [SEP]),
    # truncar/paddear a max_len y devolver tensores de PyTorch.
    encodings = tokenizer(
        list(texts),
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encodings['input_ids']
    attention_mask = encodings['attention_mask']
    labels_tensor = torch.tensor(labels)

    dataset = TensorDataset(input_ids, attention_mask, labels_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

# Se crearán los DataLoaders específicos para cada modelo, ya que usan diferentes tokenizers.


--- Cargando y Preprocesando el Dataset IMDB ---
Dataset IMDB cargado.
Ejemplos de entrenamiento: 25000
Ejemplos de prueba: 25000
Ejemplo de texto: 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it w...'
Etiqueta correspondiente: Negativo


In [5]:
# 4. Definición de los Modelos (BERT y DistilBERT)
# ------------------------------------------------------------------------------
# Los modelos se cargarán directamente desde la librería `transformers`.

# Modelo 1: DistilBERT (más ligero y rápido)
print("\n--- Cargando Tokenizer y Modelo DistilBERT ---")
tokenizer_distilbert = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_distilbert = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2, # 2 clases: positivo y negativo
    output_attentions=False,
    output_hidden_states=False,
)
print("DistilBERT cargado.")

# Modelo 2: BERT (más grande y potencialmente más preciso)
print("\n--- Cargando Tokenizer y Modelo BERT ---")
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
)
print("BERT cargado.")


--- Cargando Tokenizer y Modelo DistilBERT ---


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT cargado.


In [6]:
# 5. Definición de las Funciones de Entrenamiento y Evaluación
# ------------------------------------------------------------------------------

def train_one_epoch(model, dataloader, optimizer, scheduler, device, epoch_num, total_epochs):
    """Realiza una época de entrenamiento para un modelo de Transformers."""
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch_num+1}/{total_epochs} [Training]")

    for batch in progress_bar:
        batch = tuple(b.to(device) for b in batch)
        b_input_ids, b_attention_mask, b_labels = batch

        model.zero_grad()

        # Prepare model inputs, conditionally including token_type_ids for BERT
        inputs = {
            'input_ids': b_input_ids,
            'attention_mask': b_attention_mask,
            'labels': b_labels
        }
        # Check if the model accepts token_type_ids (like BERT does)
        if isinstance(model, BertForSequenceClassification):
             inputs['token_type_ids'] = None # Or your specific token_type_ids if needed


        # El forward pass devuelve la pérdida directamente cuando se proporcionan etiquetas
        outputs = model(**inputs) # Use **inputs to unpack the dictionary

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Prevenir "exploding gradients"
        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': f'{loss.item():.3f}'})

    avg_train_loss = total_loss / len(dataloader)
    print(f"  Avg. training loss: {avg_train_loss:.4f}")
    return avg_train_loss


def evaluate_model(model, dataloader, device):
    """Evalúa el modelo en un conjunto de datos (validación/prueba)."""
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    progress_bar = tqdm(dataloader, desc="Evaluating")

    for batch in progress_bar:
        batch = tuple(b.to(device) for b in batch)
        b_input_ids, b_attention_mask, b_labels = batch

        with torch.no_grad():
            # Prepare model inputs, conditionally including token_type_ids for BERT
            inputs = {
                'input_ids': b_input_ids,
                'attention_mask': b_attention_mask,
                'labels': b_labels
            }
            # Check if the model accepts token_type_ids (like BERT does)
            if isinstance(model, BertForSequenceClassification):
                inputs['token_type_ids'] = None # Or your specific token_type_ids if needed


            outputs = model(**inputs) # Use **inputs to unpack the dictionary

        loss = outputs.loss
        logits = outputs.logits

        total_eval_loss += loss.item()

        # Mover logits y etiquetas a la CPU para cálculo de precisión
        logits = logits.detach().cpu().numpy()
        labels = b_labels.to('cpu').numpy()

        preds = np.argmax(logits, axis=1).flatten()
        total_eval_accuracy += np.sum(preds == labels)

    avg_val_accuracy = total_eval_accuracy / len(dataloader.dataset)
    avg_val_loss = total_eval_loss / len(dataloader)
    print(f"  Accuracy: {avg_val_accuracy:.4f}")
    print(f"  Validation loss: {avg_val_loss:.4f}")
    return avg_val_accuracy, avg_val_loss


def train_and_evaluate_transformer(model, tokenizer, model_name):
    """Bucle principal de entrenamiento y evaluación."""
    print(f"\n--- Entrenando y Evaluando: {model_name} en {device} ---")

    # Crear DataLoaders para este modelo específico
    # Ensure train_texts and train_labels are lists
    train_dataloader = create_dataloader(list(train_texts), list(train_labels), tokenizer, MAX_LEN, BATCH_SIZE)
    test_dataloader = create_dataloader(list(test_texts), list(test_labels), tokenizer, MAX_LEN, BATCH_SIZE)


    model.to(device)

    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)
    total_steps = len(train_dataloader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=total_steps)

    best_val_acc = 0.0
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}

    for epoch in range(EPOCHS):
        print(f"\nIniciando Época {epoch+1}/{EPOCHS}")
        train_loss = train_one_epoch(model, train_dataloader, optimizer, scheduler, device, epoch, EPOCHS)
        val_acc, val_loss = evaluate_model(model, test_dataloader, device)

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            # Guardamos el estado del mejor modelo provisionalmente
            torch.save(model.state_dict(), f'{model_name.replace(" ", "_")}_best_temp.pth')
            print(f"  Nueva mejor Acc Validación: {best_val_acc:.4f} (guardado provisionalmente)")

    print(f"\nMejor resultado para {model_name}: Acc Val = {best_val_acc:.4f}")
    # Cargamos el mejor estado guardado
    model.load_state_dict(torch.load(f'{model_name.replace(" ", "_")}_best_temp.pth'))
    os.remove(f'{model_name.replace(" ", "_")}_best_temp.pth') # Limpiamos el archivo temporal

    return model, history, best_val_acc

In [7]:
# 6. Entrenamiento y Evaluación de los Modelos
# ------------------------------------------------------------------------------

# -- Modelo 1: DistilBERT --
model_distilbert_trained, hist_distilbert, acc_distilbert = train_and_evaluate_transformer(
    model_distilbert, tokenizer_distilbert, "DistilBERT"
)

# -- Modelo 2: BERT --
# (Para una comparación justa, deberíamos usar una instancia nueva del modelo si no lo hacemos)
model_bert_trained, hist_bert, acc_bert = train_and_evaluate_transformer(
    model_bert, tokenizer_bert, "BERT"
)

# Resumen de Resultados
print("\n\n--- Resumen Final de Resultados ---")
print(f"DistilBERT: Mejor Precisión en Validación = {acc_distilbert*100:.2f}%")
print(f"BERT:       Mejor Precisión en Validación = {acc_bert*100:.2f}%")

# Seleccionar el mejor modelo
if acc_bert > acc_distilbert:
    mejor_modelo = model_bert_trained
    mejor_tokenizer = tokenizer_bert
    nombre_mejor_modelo = "BERT"
    acc_mejor_modelo = acc_bert
else:
    mejor_modelo = model_distilbert_trained
    mejor_tokenizer = tokenizer_distilbert
    nombre_mejor_modelo = "DistilBERT"
    acc_mejor_modelo = acc_distilbert

print(f"\nEl mejor modelo es: {nombre_mejor_modelo} con una precisión de {acc_mejor_modelo*100:.2f}%")


--- Entrenando y Evaluando: DistilBERT en cuda ---

Tokenizando datos con DistilBertTokenizer...

Tokenizando datos con DistilBertTokenizer...

Iniciando Época 1/3


Epoch 1/3 [Training]:   0%|          | 0/1563 [00:00<?, ?it/s]

  Avg. training loss: 0.2926


Evaluating:   0%|          | 0/1563 [00:00<?, ?it/s]

  Accuracy: 0.9011
  Validation loss: 0.2476
  Nueva mejor Acc Validación: 0.9011 (guardado provisionalmente)

Iniciando Época 2/3


...

  Accuracy: 0.9202
  Validation loss: 0.2659
  Nueva mejor Acc Validación: 0.9202 (guardado provisionalmente)

Iniciando Época 3/3


Epoch 3/3 [Training]:   0%|          | 0/1563 [00:00<?, ?it/s]

  Avg. training loss: 0.0905


Evaluating:   0%|          | 0/1563 [00:00<?, ?it/s]

  Accuracy: 0.9222
  Validation loss: 0.3365
  Nueva mejor Acc Validación: 0.9222 (guardado provisionalmente)

Mejor resultado para BERT: Acc Val = 0.9222


--- Resumen Final de Resultados ---
DistilBERT: Mejor Precisión en Validación = 91.22%
BERT:       Mejor Precisión en Validación = 92.22%

El mejor modelo es: BERT con una precisión de 92.22%


In [8]:
# 7. Inferencia con el Mejor Modelo
# ------------------------------------------------------------------------------
print(f"\n--- Inferencia Usando el modelo: {nombre_mejor_modelo} ---")

mejor_modelo.eval()
mejor_modelo.to(device)

def predecir_sentimiento(texto, modelo, tokenizer, device, max_len=MAX_LEN):
    """Realiza una predicción para un único texto."""
    encoded_review = tokenizer.encode_plus(
        texto,
        max_length=max_len,
        add_special_tokens=True,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    with torch.no_grad():
        outputs = modelo(input_ids, attention_mask=attention_mask)

    probs = torch.softmax(outputs.logits, dim=1).cpu().numpy()[0]
    pred_label_idx = np.argmax(probs)
    pred_confidence = probs[pred_label_idx]

    clases = ['Negativo', 'Positivo']
    return clases[pred_label_idx], pred_confidence

# Ejemplos de inferencia
reviews_de_prueba = [
    "This movie was absolutely fantastic! The acting was superb and the plot was gripping.",
    "I would not recommend this film to anyone. It was boring, slow, and poorly directed.",
    "A decent movie, not the best I've seen, but certainly not the worst either.",
    "The visual effects were stunning, but the story was very weak and predictable."
]

for review in reviews_de_prueba:
    sentimiento, confianza = predecir_sentimiento(review, mejor_modelo, mejor_tokenizer, device)
    print(f"\nReview: '{review}'")
    print(f"  -> Predicción: {sentimiento} (Confianza: {confianza:.2%})")


--- Inferencia Usando el modelo: BERT ---

Review: 'This movie was absolutely fantastic! The acting was superb and the plot was gripping.'
  -> Predicción: Positivo (Confianza: 99.90%)

Review: 'I would not recommend this film to anyone. It was boring, slow, and poorly directed.'
  -> Predicción: Negativo (Confianza: 99.92%)

Review: 'A decent movie, not the best I've seen, but certainly not the worst either.'
  -> Predicción: Positivo (Confianza: 55.06%)

Review: 'The visual effects were stunning, but the story was very weak and predictable.'
  -> Predicción: Negativo (Confianza: 99.75%)


In [9]:
# 8. Guardar el Mejor Modelo
# ------------------------------------------------------------------------------

modelos_guardados_dir = "modelos_transformers_guardados"
if not os.path.exists(modelos_guardados_dir):
    os.makedirs(modelos_guardados_dir)

timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
nombre_archivo_modelo = f"{nombre_mejor_modelo.lower()}_{timestamp}.pth"
ruta_guardado_modelo = os.path.join(modelos_guardados_dir, nombre_archivo_modelo)

print(f"\nGuardando el state_dict del mejor modelo ({nombre_mejor_modelo}) en: {ruta_guardado_modelo}")
torch.save(mejor_modelo.state_dict(), ruta_guardado_modelo)
print("State_dict del modelo guardado exitosamente.")

# Para guardar el modelo completo y el tokenizer (forma recomendada por Hugging Face)
directorio_guardado_completo = os.path.join(modelos_guardados_dir, f"{nombre_mejor_modelo.lower()}_{timestamp}_completo")
mejor_modelo.save_pretrained(directorio_guardado_completo)
mejor_tokenizer.save_pretrained(directorio_guardado_completo)
print(f"Modelo y tokenizer guardados en el directorio: {directorio_guardado_completo}")


Guardando el state_dict del mejor modelo (BERT) en: modelos_transformers_guardados/bert_20250608-195534.pth
State_dict del modelo guardado exitosamente.
Modelo y tokenizer guardados en el directorio: modelos_transformers_guardados/bert_20250608-195534_completo


## **Conclusiones y Próximos Pasos**

En este script, hemos seguido un flujo de trabajo para la clasificación de texto con Transformers:
1.  Cargamos el dataset IMDB usando `torchtext`.
2.  Tokenizamos los datos usando los tokenizers específicos de `DistilBERT` y `BERT`.
3.  Creamos `DataLoader` para manejar los lotes de datos eficientemente.
4.  Cargamos los modelos pre-entrenados `DistilBertForSequenceClassification` y `BertForSequenceClassification`.
5.  Implementamos un bucle de entrenamiento y evaluación optimizado para los modelos de Hugging Face.
6.  Comparamos el rendimiento de ambos modelos, realizamos inferencias y guardamos el mejor.

**Comparación y Próximos Pasos:**
 * **DistilBERT vs BERT:** Notarás que DistilBERT entrena significativamente más rápido y consume menos recursos, mientras que BERT puede ofrecer una ligera mejora en la precisión a un costo computacional mayor.
 * **Ajuste de Hiperparámetros:** Experimentar con `LEARNING_RATE`, `BATCH_SIZE`, `MAX_LEN` y el número de `EPOCHS` puede mejorar los resultados.
 * **Otros Modelos:** Probar con otros modelos de la librería `transformers` como RoBERTa o ALBERT.
 * **Planificador de Tasa de Aprendizaje (`scheduler`):** Usamos un `get_linear_schedule_with_warmup`, que es una práctica estándar. Experimentar con otros schedulers o diferentes números de `warmup_steps` podría ser beneficioso.